In [ ]:
%matplotlib inline

from matplotlib import rcParams

rcParams["figure.dpi"] = 300
rcParams["font.size"] = 8

import warnings

warnings.filterwarnings("ignore")

# Create 2D integer masks

In this tutorial we will show how to create 2D integer mask for arbitrary latitude and longitude grids.

Import regionmask and check the version:

In [ ]:
import regionmask

regionmask.__version__

Load xarray and the tutorial data:

In [ ]:
import xarray as xr
import numpy as np

xr.set_options(display_style="text", display_width=60)

## Creating a mask

Define a lon/ lat grid with a 1° grid spacing, where the points define the center of the grid.

In [ ]:
lon = np.arange(-179.5, 180)
lat = np.arange(-89.5, 90)

We will create a mask with the SREX regions (Seneviratne et al., 2012).

In [ ]:
regionmask.defined_regions.srex

The function `mask` determines which gridpoints lie within the polygon making up the each region:

In [ ]:
mask = regionmask.defined_regions.srex.mask(lon, lat)
mask

`mask` is now a `xarray.Dataset` with shape `lat x lon` (if you need a numpy array use `mask.values`). Gridpoints that do not fall in a region are `NaN`, the gridpoints that fall in a region are encoded with the number of the region (here 1 to 26).

We can now plot the `mask`:

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

regionmask.defined_regions.srex.plot(
    ax=ax, add_label=False, line_kws=dict(lw=0.5, color="0.5")
)

mask.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False)

## Working with a mask

masks can be used to select data in a certain region and to calculate regional averages - let's illustrate this with a 'real' dataset:

In [ ]:
airtemps = xr.tutorial.load_dataset("air_temperature")

The example data is a temperature field over North America. Let's plot the first time step:

In [ ]:
# choose a good projection for regional maps
proj = ccrs.LambertConformal(central_longitude=-100)

ax = plt.subplot(111, projection=proj)

airtemps.isel(time=1).air.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree())

ax.coastlines();

Conveniently we can directly pass an xarray object to the `mask` function. It gets the longitude and latitude from the `DataArray`/ `Dataset` and creates the `mask`. Per default regionmask assumes the longitude and latitude are called `"lon"` and `"lat"`. If they have another name, you can pass them individually, e.g. `region.mask(ds.longitude, ds.latitude)`.

In [ ]:
mask = regionmask.defined_regions.srex.mask(airtemps)

In [ ]:
lon = airtemps.lon.values
print("Grid extent:    {:3.0f}°E to {:3.0f}°E".format(lon.min(), lon.max()))

bounds = regionmask.defined_regions.srex.bounds_global
print("Region extent: {:3.0f}°E to {:3.0f}°E".format(bounds[0], bounds[2]))

Let's plot the mask of the regions:

In [ ]:
proj = ccrs.LambertConformal(central_longitude=-100)
ax = plt.subplot(111, projection=proj)

low = mask.min()
high = mask.max()

levels = np.arange(low - 0.5, high + 1)

h = mask.plot.pcolormesh(
    ax=ax, transform=ccrs.PlateCarree(), levels=levels, add_colorbar=False
)

# for colorbar: find abbreviations of all regions that were selected
reg = np.unique(mask.values)
reg = reg[~np.isnan(reg)]
abbrevs = regionmask.defined_regions.srex[reg].abbrevs

cbar = plt.colorbar(h, orientation="horizontal", fraction=0.075, pad=0.05)

cbar.set_ticks(reg)
cbar.set_ticklabels(abbrevs)
cbar.set_label("Region")

ax.coastlines()

# fine tune the extent
ax.set_extent([200, 330, 10, 75], crs=ccrs.PlateCarree())

### Select a specific region - using cf_xarray

If [cf_xarray](https://cf-xarray.readthedocs.io/en/latest/index.html) is installed and imported we can select any individual region using the [flags](https://cf-xarray.readthedocs.io/en/latest/flags.html) saved in the attributes (`attrs`) of the mask (new in regionmask 0.10.0):

In [ ]:
import cf_xarray

In [ ]:
mask

Thus to select the region named "Central North America" (abbreviated "CNA") we can do:

In [ ]:
mask_CNA = mask.cf == "CNA"

# show a subset
mask_CNA.sel(lat=30, lon=slice(240, 260))

In [ ]:
mask_names = regionmask.defined_regions.srex.mask(airtemps, flag="names")

try:
    mask_names.cf == "C. North America"
except ValueError:
    print("error raised")

mask_names.cf == "C. North America".replace(" ", "_")

### Select a specific region - without cf_xarray

We first need to find out which number the region 'C. North America' corresponds to:

In [ ]:
CNA_index = regionmask.defined_regions.srex.map_keys("C. North America")
CNA_index

In [ ]:
mask_CNA = mask == CNA_index

### Mask out a region

To replace all grid points outside of CNA we use the `where` method of `xr.Dataset` ([documentation](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.where.html)), which filters elements from this object according to a condition:

In [ ]:
airtemps_CNA = airtemps.where(mask_CNA)

Check everything went well by repeating the first plot with the selected region:

In [ ]:
# choose a good projection for regional maps
proj = ccrs.LambertConformal(central_longitude=-100)

ax = plt.subplot(111, projection=proj)

regionmask.defined_regions.srex[["CNA"]].plot(ax=ax, add_label=False)

airtemps_CNA.isel(time=1).air.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree())

ax.coastlines();

Looks good - with this we can calculate the region average.

### Calculate weighted regional average

From version 0.15.1 xarray includes a function to calculate the weighted mean - we use `cos(lat)` as proxy of the grid cell area

In [ ]:
weights = np.cos(np.deg2rad(airtemps.lat))

ts_airtemps_CNA = airtemps_CNA.weighted(weights).mean(dim=("lat", "lon")) - 273.15

We plot the resulting time series:

In [ ]:
f, ax = plt.subplots()
ts_airtemps_CNA.air.plot.line(ax=ax, label="Central North America")

ax.axhline(0, color="0.1", lw=0.5)

plt.legend();

To get the regional average for each region you would need to loop over them. However, it's easier to use a 3D mask.

### Calculate regional statistics using `groupby`

In [ ]:
# you can group over all integer values of the mask
airtemps_all = airtemps.groupby(mask).mean()
airtemps_all

However, `groupby` is the way to go when calculating a (unweighted) regional median:

In [ ]:
# you can group over all integer values of the mask
airtemps_reg_median = airtemps.groupby(mask).median()
airtemps_reg_median.isel(time=0)

## Multidimensional coordinates

Regionmask can also handle mutltidimensional longitude/ latitude grids (e.g. from a regional climate model). As xarray provides such an example dataset, we will use it to illustrate it. See also in the [xarray documentation](http://xarray.pydata.org/en/stable/examples/multidimensional-coords.html/).

Load the tutorial data:

In [ ]:
rasm = xr.tutorial.load_dataset("rasm")

The example data is a temperature field over the Northern Hemisphere. Let's plot the first time step:

In [ ]:
# choose a projection
proj = ccrs.NorthPolarStereo()

ax = plt.subplot(111, projection=proj)
ax.set_global()

rasm.isel(time=1).Tair.plot.pcolormesh(
    ax=ax, x="xc", y="yc", transform=ccrs.PlateCarree()
)

# add the abbreviation of the regions
regionmask.defined_regions.srex[[1, 2, 11, 12, 18]].plot(
    ax=ax, add_coastlines=False, label="abbrev"
)

ax.set_extent([-180, 180, 43, 90], ccrs.PlateCarree())

ax.coastlines();

Again we pass the xarray object to regionmask. Here, cf_xarray is used to detect `rasm.xc` and `rasm.yc` as longitude and latitude coordinates (new in regionmask 0.10.0). Without cf_xarray we would need to pass the coordinates explicitely (i.e. `srex.mask(rasm.xc, rasm.yc`):

In [ ]:
mask = regionmask.defined_regions.srex.mask(rasm)
mask

We want to select the region 'NAS' (Northern Asia).

### Select using `where`

Using the cf_xarray accessor (from regionmask 0.10.0) we can directly use the name of the region (otherwise we need to select by the the number of the region):

In [ ]:
rasm_NAS = rasm.where(mask.cf == "NAS")

Check everything went well by repeating the first plot with the selected region:

In [ ]:
# choose a projection
proj = ccrs.NorthPolarStereo()

ax = plt.subplot(111, projection=proj)
ax.set_global()

rasm_NAS.isel(time=1).Tair.plot.pcolormesh(
    ax=ax, x="xc", y="yc", transform=ccrs.PlateCarree()
)


# add the abbreviation of the regions
regionmask.defined_regions.srex[["NAS"]].plot(
    ax=ax, add_coastlines=False, label="abbrev"
)

ax.set_extent([-180, 180, 45, 90], ccrs.PlateCarree())

ax.coastlines();

## References

* Special Report on Managing the Risks of Extreme Events and Disasters to Advance Climate Change Adaptation (SREX, Seneviratne et al., [2012](https://www.ipcc.ch/site/assets/uploads/2018/03/SREX-Ch3-Supplement_FINAL-1.pdf))